# Event/DLC/DA Alignment Utilities

This notebook provides reusable functions to:
- Load event CSVs (camera-strobe aligned)
- Load DeepLabCut (DLC) CSVs (60 Hz) and compute kinematics
- Load DA (TDT) dFF and strobe
- Load Neuropixels strobe
- Compute alignment between strobe signals (DA ↔ NP ↔ camera) so everything shares a common time base

Paths are prefilled for the 1818 dataset. Edit them if needed.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import correlate
from tqdm.auto import tqdm
from scipy.signal import find_peaks

# ---- paths (edit as needed) ----
event_csvs = [
    Path(r"Z:\Koji\NP_Coh3\Recording\Day27_1818_Clockwise_corner_2025-11-20T15_11_30.csv"),
    Path(r"Z:\Koji\NP_Coh3\Recording\Day27_1818_Clockwise_licking_2025-11-20T15_11_30.csv"),
]
dlc_csv = Path(r"Z:\Koji\NP_Coh3\Recording\Day27_1818_Clockwise2025-11-20T15_50_10DLC_HrnetW32_openfield_v3Sep10shuffle2_detector_170_snapshot_160.csv")
da_mat = Path(r"Z:\Koji\NP_Coh3\TDT\1818\250919\1818_250919_dFF.mat")
np_strobe_npy = Path(r"Z:\Koji\Neuropixels\1818\1818_11202025_g0\1818_11202025_g0_imec0\kilosort4\strobe_signal.npy")  # derive per SpikeSorting notebook

# camera frame rate
cam_fps = 60.0

print("Paths set. Edit above if needed.")

In [ ]:
# Load events CSVs and merge
def load_events(csv_paths):
    dfs = []
    for p in csv_paths:
        df = pd.read_csv(p)
        df["source_file"] = p.name
        dfs.append(df)
    ev = pd.concat(dfs, ignore_index=True)
    return ev

events = load_events(event_csvs)
print("Events loaded:", events.shape)
display(events.head())

In [ ]:
# Load DLC CSV (DeepLabCut)
dlc = pd.read_csv(dlc_csv, header=None)
# DLC formats vary; here assume standard DLC wide CSV with scorer/bodyparts likelihood triplets.
# Extract time vector based on cam_fps.
dlc_time = np.arange(len(dlc)) / cam_fps
print("DLC loaded:", dlc.shape, "time span (s)", dlc_time[-1])

In [ ]:
# Load DA dFF and strobe from TDT mat
da = loadmat(da_mat)
# Inspect keys to locate signals
print("DA mat keys:", da.keys())
# Example: dFF signal under key 'dff', strobe under 'strobe', sample rate under 'fs'
da_dff = None
da_strobe = None
da_fs = None
for k in da.keys():
    if k.lower() == 'dff':
        da_dff = np.asarray(da[k]).squeeze()
    if 'strobe' in k.lower():
        da_strobe = np.asarray(da[k]).squeeze()
    if k.lower() == 'fs':
        da_fs = float(np.asarray(da[k]).squeeze())
print("DA signals: dff", None if da_dff is None else da_dff.shape, "strobe", None if da_strobe is None else da_strobe.shape, "fs", da_fs)
da_time = np.arange(len(da_dff)) / da_fs if da_dff is not None else None

In [ ]:
# Load Neuropixels strobe (pre-extracted). If not available, extract per SpikeSorting.ipynb guidance.
np_strobe = np.load(np_strobe_npy) if np_strobe_npy.exists() else None
print("NP strobe shape:", None if np_strobe is None else np_strobe.shape)

In [ ]:
# Alignment helpers: detect strobe edges and build linear mapping between time bases
def detect_strobe_edges(sig, fs, height=None, distance=None):
    """Return edge times (seconds) from a TTL-like strobe signal."""
    sig = np.asarray(sig).astype(float)
    if height is None:
        height = 0.5 * (sig.min() + sig.max())
    if distance is None:
        distance = max(1, int(0.5 * fs / cam_fps))  # at least ~half a frame apart
    peaks, _ = find_peaks(sig, height=height, distance=distance)
    return peaks / fs

def fit_time_mapping(src_times, dst_times):
    """Linear mapping src->dst: dst = a*src + b using least squares."""
    src = np.asarray(src)
    dst = np.asarray(dst)
    a, b = np.polyfit(src, dst, 1)
    return a, b

def apply_mapping(times, a, b):
    return a * np.asarray(times) + b

# Build mappings if strobes are available
mapping_cam_to_np = None
mapping_da_to_np = None

if np_strobe is not None:
    # NP strobe edges (assume fs from LFP)
    fs_np = None
    try:
        import spikeinterface.extractors as se
        # fallback to using a small NP recording if needed; else assume 30k/1k depending on stream
    except Exception:
        pass
    # If you know NP strobe fs, set fs_np here; otherwise assume 30000 for AP or 1000 for LF
    fs_np = 1000.0
    np_edges = detect_strobe_edges(np_strobe, fs=fs_np)

    # Camera frame times (60 Hz) in camera time base
    cam_times = np.arange(len(dlc_time)) / cam_fps if 'dlc_time' in locals() else None

    if cam_times is not None and len(np_edges) >= len(cam_times):
        # Align first len(cam_times) edges
        cam_subset = cam_times
        np_subset = np_edges[:len(cam_subset)]
        a, b = fit_time_mapping(cam_subset, np_subset)
        mapping_cam_to_np = (a, b)
        print(f"Camera->NP mapping: t_np = {a:.6f} * t_cam + {b:.6f}")
    else:
        print("Not enough NP strobe edges to map camera frames; check strobe extraction.")

    if da_strobe is not None and da_fs is not None:
        da_edges = detect_strobe_edges(da_strobe, fs=da_fs)
        n_match = min(len(da_edges), len(np_edges))
        if n_match > 5:
            a, b = fit_time_mapping(da_edges[:n_match], np_edges[:n_match])
            mapping_da_to_np = (a, b)
            print(f"DA->NP mapping: t_np = {a:.6f} * t_da + {b:.6f}")
        else:
            print("Not enough overlapping strobe edges for DA->NP mapping.")
else:
    print("NP strobe missing; cannot build mappings. Extract NP strobe per SpikeSorting.ipynb.")

In [ ]:
# Helper converters using fitted mappings
def cam_to_np_time(cam_times):
    if mapping_cam_to_np is None:
        raise RuntimeError("Camera->NP mapping not available")
    a, b = mapping_cam_to_np
    return apply_mapping(cam_times, a, b)

def da_to_np_time(da_times):
    if mapping_da_to_np is None:
        raise RuntimeError("DA->NP mapping not available")
    a, b = mapping_da_to_np
    return apply_mapping(da_times, a, b)

# Example: align events and DLC to NP time
if mapping_cam_to_np is not None:
    cam_times = np.arange(len(dlc_time)) / cam_fps
    dlc_time_np = cam_to_np_time(cam_times)
    print("DLC time (NP base):", dlc_time_np[:5], "...", dlc_time_np[-5:])
    # If events have a 'frame' or 'time_cam' column, map it similarly:
    # events['time_np'] = cam_to_np_time(events['frame'] / cam_fps)
else:
    print("Camera->NP mapping unavailable; cannot map DLC/events yet.")

print("Alignment helpers ready. Verify strobe extraction in DA_analysis.ipynb and SpikeSorting.ipynb if mappings fail.")